# Trabajo Práctico Final: Linear/Quadratic Discriminant Analysis (LDA/QDA)

### Definición: Clasificador Bayesiano

Sean $k$ poblaciones, $x \in \mathbb{R}^p$ puede pertenecer a cualquiera $g \in \mathcal{G}$ de ellas. Bajo un esquema bayesiano, se define entonces $\pi_j \doteq P(G = j)$ la probabilidad *a priori* de que $X$ pertenezca a la clase *j*, y se **asume conocida** la distribución condicional de cada observable dado su clase $f_j \doteq f_{X|G=j}$.

De esta manera dicha probabilidad *a posteriori* resulta
$$
P(G|_{X=x} = j) = \frac{f_{X|G=j}(x) \cdot p_G(j)}{f_X(x)} \propto f_j(x) \cdot \pi_j
$$

La regla de decisión de Bayes es entonces
$$
H(x) \doteq \arg \max_{g \in \mathcal{G}} \{ P(G|_{X=x} = j) \} = \arg \max_{g \in \mathcal{G}} \{ f_j(x) \cdot \pi_j \}
$$

es decir, se predice a $x$ como perteneciente a la población $j$ cuya probabilidad a posteriori es máxima.

*Ojo, a no desesperar! $\pi_j$ no es otra cosa que una constante prefijada, y $f_j$ es, en su esencia, un campo escalar de $x$ a simplemente evaluar.*

### Distribución condicional

Para los clasificadores de discriminante cuadrático y lineal (QDA/LDA) se asume que $X|_{G=j} \sim \mathcal{N}_p(\mu_j, \Sigma_j)$, es decir, se asume que cada población sigue una distribución normal.

Por definición, se tiene entonces que para una clase $j$:
$$
f_j(x) = \frac{1}{(2 \pi)^\frac{p}{2} \cdot |\Sigma_j|^\frac{1}{2}} e^{- \frac{1}{2}(x-\mu_j)^T \Sigma_j^{-1} (x- \mu_j)}
$$

Aplicando logaritmo (que al ser una función estrictamente creciente no afecta el cálculo de máximos/mínimos), queda algo mucho más práctico de trabajar:

$$
\log{f_j(x)} = -\frac{1}{2}\log |\Sigma_j| - \frac{1}{2} (x-\mu_j)^T \Sigma_j^{-1} (x- \mu_j) + C
$$

Observar que en este caso $C=-\frac{p}{2} \log(2\pi)$, pero no se tiene en cuenta ya que al tener una constante aditiva en todas las clases, no afecta al cálculo del máximo.

### LDA

En el caso de LDA se hace una suposición extra, que es $X|_{G=j} \sim \mathcal{N}_p(\mu_j, \Sigma)$, es decir que las poblaciones no sólo siguen una distribución normal sino que son de igual matriz de covarianzas. Reemplazando arriba se obtiene entonces:

$$
\log{f_j(x)} =  -\frac{1}{2}\log |\Sigma| - \frac{1}{2} (x-\mu_j)^T \Sigma^{-1} (x- \mu_j) + C
$$

Ahora, como $-\frac{1}{2}\log |\Sigma|$ es común a todas las clases se puede incorporar a la constante aditiva y, distribuyendo y reagrupando términos sobre $(x-\mu_j)^T \Sigma^{-1} (x- \mu_j)$ se obtiene finalmente:

$$
\log{f_j(x)} =  \mu_j^T \Sigma^{-1} (x- \frac{1}{2} \mu_j) + C'
$$

### Entrenamiento/Ajuste

Obsérvese que para ambos modelos, ajustarlos a los datos implica estimar los parámetros $(\mu_j, \Sigma_j) \; \forall j = 1, \dots, k$ en el caso de QDA, y $(\mu_j, \Sigma)$ para LDA.

Estos parámetros se estiman por máxima verosimilitud, de manera que los estimadores resultan:

* $\hat{\mu}_j = \bar{x}_j$ el promedio de los $x$ de la clase *j*
* $\hat{\Sigma}_j = s^2_j$ la matriz de covarianzas estimada para cada clase *j*
* $\hat{\pi}_j = f_{R_j} = \frac{n_j}{n}$ la frecuencia relativa de la clase *j* en la muestra
* $\hat{\Sigma} = \frac{1}{n} \sum_{j=1}^k n_j \cdot s^2_j$ el promedio ponderado (por frecs. relativas) de las matrices de covarianzas de todas las clases. *Observar que se utiliza el estimador de MV y no el insesgado*

Es importante notar que si bien todos los $\mu, \Sigma$ deben ser estimados, la distribución *a priori* puede no inferirse de los datos sino asumirse previamente, utilizándose como entrada del modelo.

### Predicción

Para estos modelos, al igual que para cualquier clasificador Bayesiano del tipo antes visto, la estimación de la clase es por método *plug-in* sobre la regla de decisión $H(x)$, es decir devolver la clase que maximiza $\hat{f}_j(x) \cdot \hat{\pi}_j$, o lo que es lo mismo $\log\hat{f}_j(x) + \log\hat{\pi}_j$.

## Estructura del código

## Modelo 

In [1]:
import numpy as np
from numpy.linalg import det, inv

In [2]:
class ClassEncoder:
  def fit(self, y):
    self.names = np.unique(y) #Obtengo los nombres unicos de la matriz y
    #print(self.names)
    #salida: ['setosa' 'versicolor' 'virginica']
    
    self.name_to_class = {name:idx for idx, name in enumerate(self.names)} #crea un diccionario, donde asignamos a cada 
    #nombre de los target, su número con la funcion enumerate. 
    #print(self.name_to_class)
    #salida: {'setosa': 0, 'versicolor': 1, 'virginica': 2}
    self.fmt = y.dtype #crea un array del mismo tipo de dato que y (str) que usará mas tarde 
    
    # Q1: why is there no need for class_to_name?
    
    """ 
    Al ejecutar la funcion get_iris_data() armamos a la matriz de target con los nombres de las flores para una mejor lectura de 
    los datos, pero al momento de trabajar con esa matriz necesitamos convertirla a numeros para mejorar la eficiencia del
    algoritmo al tener que comparar las clases.

    """

  def _map_reshape(self, f, arr):
    
    return np.array([f(elem) for elem in arr.flatten()]).reshape(arr.shape)

    # Q2: why is reshaping necessary?
    """
    Esta función recibe la matriz de etiquetas y (1,90) y nos devuelve devuelve la matriz de etiquetas (1,90) pero 
    convertida a nros.
    f tiene la información de que valor le corresponde a cada etiqueta, entonces se hace una iteración sobre el array 
    de etiquetas que es arr.flatten me quedo con el valor numerico. 
    Luego de esto tenemos como resultado un arreglo con 90 elementos, pero nosotros necesitamos el vector columna
    transformado, entoconces realizaomos un reshape para volver a tenerlo.
    
    La función flatten() se usa para obtener una copia de una matriz dada colapsada en una dimensión, entonces 
    a la matriz arr que se recibe como parametro se la convierte a una dimensión. Esto se realiza para que sea 
    genérico el metodo. Al fin y al cabo lo que se quiere hacer es un encoder de los nombres, no cambiar morfología. 

    """
  def transform(self, y):
    return self._map_reshape(lambda name: self.name_to_class[name], y) 

  def fit_transform(self, y):
    self.fit(y)
    return self.transform(y)

  def detransform(self, y_hat):# Esta funcion hace la inversa, si le paso un vector con 0,1,2 te devuelve los nombres. 
    return self._map_reshape(lambda idx: self.names[idx], y_hat)

In [3]:
class BaseBayesianClassifier:
  def __init__(self):
    self.encoder = ClassEncoder() 

  def _estimate_a_priori(self, y):
    a_priori = np.bincount(y.flatten().astype(int)) / y.size
    
    # Q3: what does bincount do?
    """
    Esta función recibe la matriz y ya codificada con los valores de cada clase y lo que hace con bicount
    es contar el numero de ocurrencias de cada valor único en el array. Luego se divide por la longitud de y, 
    entonces la variable a_priori contiene las frecuencias relativas de cada clase en el dataset.  
    Saca basicamente lo que vendría a ser un histograma.
    
    """
    return np.log(a_priori)
    
  def _fit_params(self, X, y):
    # estimate all needed parameters for given model
    raise NotImplementedError()

  def _predict_log_conditional(self, x, class_idx):
    # predict the log(P(x|G=class_idx)), the log of the conditional probability of x given the class
    # this should depend on the model used
    raise NotImplementedError()

  def fit(self, X, y, a_priori=None): #esta es la función que recibe los datos de train_X train_y
    # first encode the classes
    y = self.encoder.fit_transform(y) #codifica
    
    # if it's needed, estimate a priori probabilities
    #como a priori esta seteado en none, llamo a la funcion estimar a priori y le envio como parametro la matriz codificada 
    
    self.log_a_priori = self._estimate_a_priori(y) if a_priori is None else np.log(a_priori)

    # check that a_priori has the correct number of classes
    #Debemos chequear que tengamos tantas probabilidades a priori como número de clases
    assert len(self.log_a_priori) == len(self.encoder.names), "A priori probabilities do not match number of classes"

    # now that everything else is in place, estimate all needed parameters for given model
    self._fit_params(X, y)
    
    # Q4: why do we need to do this last, can't we do it first?
    
    """
    Sin encoder no se puede indexar con la matriz y porque recibiriamos valores de tipo string
    Sin estimar a priori, esta funcion tampoco se podria evaluar len(self.log_a_priori)
    """
##############################----------------inicia predicciones---------------------########################################
  def predict(self, X):
    # this is actually an individual prediction encased in a for-loop
    
    #recibe la matriz de caracteristicas X, y guarda el numero de observaciones en m_obs. Usa [1] porque estan en columnas
    m_obs = X.shape[1]

    #Arma un array vacío del tamaño de las observaciones y del tipo string (aca se van a guardar las predicciones de las clases)
    y_hat = np.empty(m_obs, dtype=self.encoder.fmt)

    for i in range(m_obs):
        
      encoded_y_hat_i = self._predict_one(X[:,i].reshape(-1,1))
      #llama a la funcion_predict_one y le pasa como parametro la matriz
      # de observaciones X (4x90) haciendo este reshape le indicamos que sea de una 
      #sola columna pero con las mismas filas (4,1) entonces tenemos los 4 datos de una observación 
        
      y_hat[i] = self.encoder.names[encoded_y_hat_i]

    # return prediction as a row vector (matching y) 
    return y_hat.reshape(1,-1)

  def _predict_one(self, x):
    # calculate all log posteriori probabilities (actually, +C)
    log_posteriori = [ log_a_priori_i + self._predict_log_conditional(x, idx) for idx, log_a_priori_i 
                  in enumerate(self.log_a_priori)]             
    # return the class that has maximum a posteriori probability
    return np.argmax(log_posteriori)

In [4]:
class QDA(BaseBayesianClassifier):

  def _fit_params(self, X, y):
    # estimate COMMON  covariance matrix
      
    self.inv_covs = [inv(np.cov(X[:,y.flatten()==idx], bias=True))#todas las filas tal que pertenezcan a la misma clase. 
                      for idx in range(len(self.log_a_priori))] #el idx va a tomar los valores 0 1 2 
    
    # Q5: why not just X[:,y==idx]?
    
    """ la variable inv_covs es una lista que contiene tres matrices de covarianza de 4x4 (caracteristicas) en principio, distintas. 
    Para poder indexar se necesita un vector unidimensional, y el vector y tiene forma de matriz de 90x1. 
    Por lo cual se utiliza la funcion flatten, que como explicamos anteriormente se usa para obtener una copia
    de una matriz dada colapsada en una dimensión.
    """

    # Q6: what does bias=True mean? why not use bias=False?
    
    """
    Para los clasificadores de discriminante cuadrático y lineal (QDA/LDA)se asume que cada población 
    sigue una distribución normal. Y sabemos que para esos casos, el estimador de máxima verosimilitud para la 
    covarianza de poblaciones que siguen distribucion normal tiene un factor de normalizacion de tipo 1/N, donde N 
    es el número de observaciones.
    
    Bias es el sesgo, la normalización predeterminada (bias=False) es 1/(N - 1), (estimación no sesgada).
    Si bias=True entonces la normalización es 1/N.

    """
    self.means = [X[:,y.flatten()==idx].mean(axis=1, keepdims=True) 
                  for idx in range(len(self.log_a_priori))]
    
    #Se obtiene la media de las cuatro características para cada clase. Luego el vector de medias es una lista 
    #de 3 elementos donde cada elemento tiene 4 componentes
    
    # Q7: what does axis=1 mean? why not axis=0 instead?

    """ Se utiliza axis =1 para calcular la media en dirección de los renglones, porque los datos estan traspuestos(4x90). Si se
        utilizaría axis=0 la operación se realizaría en dirección a las columnas. 
   """
    
  def _predict_log_conditional(self, x, class_idx):#recibe la matriz de observaciones redimensionada (4,1) y el indice de cada clase
    # predict the log(P(x|G=class_idx)), the log of the conditional probability of x given the class
    # this should depend on the model used
    
    #Calcula el logaritmo de la probabilidad condicional de X de cada clase
    
    inv_cov = self.inv_covs[class_idx] 
    unbiased_x =  x - self.means[class_idx]

    return 0.5*np.log(det(inv_cov)) -0.5 * unbiased_x.T @ inv_cov @ unbiased_x

## Código para pruebas

In [5]:
# hiperparámetros
rng_seed = 65

In [6]:
from sklearn.datasets import load_iris 

def get_iris_dataset():
  data = load_iris()
  X_full = data.data 
  y_full = np.array([data.target_names[y] for y in data.target.reshape(-1,1)])  
  return X_full, y_full

X_full, y_full= get_iris_dataset()

"""
    data es un objeto especial de sklearn para guardar dataset y sus atributos

    X_full es matriz de 150x4 cada fila es una observación y cada columna una caracteristica (predictor)

    data.target reprenta lo que vamos a predecir, el tipo de flor. Es un array np de 150 elementos, con los valores 0,1,2.
    
    Si aplicamos reshape(-1,1) obtenemos una matriz de (150,1)
    En data.target_names tengo los nombres tres tipos de flores ['setosa','versicolor','virginica']
    entonces lo que hacemos en y_full es guardar armar una matriz 150x1 con los tres nombres de las flores. 
    
"""
print(f"X: {X_full.shape}, Y:{y_full.shape}")


X: (150, 4), Y:(150, 1)


In [7]:
np.unique(y_full)

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [8]:
# peek data matrix
X_full[:5]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2]])

In [9]:
# peek target vector
y_full[:5]

array([['setosa'],
       ['setosa'],
       ['setosa'],
       ['setosa'],
       ['setosa']], dtype='<U10')

In [10]:
# preparing data, train - test validation
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_full, y_full, test_size=0.4, random_state=rng_seed)
"""
train_test_split es una funcion de sklearn para dividir matrices de datos en dos subconjuntos, datos de entrenamiento y datos 
de prueba. Con el parametro test=0.4 estamos pidiendo que el 40% del dataset se utilice para la prueba y el 60% para el
entrenamiento. Luego con el parametro random_state estamos pidiendo que esa separacion se realice aleatoriamente pero 
utilizando una semilla para poder obtener nuevamente la misma mezcla.
"""
# traspose everything because this model format assumes column vectors

train_x = X_train.T
train_y = y_train.T
test_x = X_test.T
test_y = y_test.T

print("Sin trasponer",X_train.shape, y_train.shape, X_test.shape, y_test.shape)
print("Traspuestas",train_x.shape, train_y.shape, test_x.shape, test_y.shape)

Sin trasponer (90, 4) (90, 1) (60, 4) (60, 1)
Traspuestas (4, 90) (1, 90) (4, 60) (1, 60)


In [11]:
qda = QDA()

qda.fit(train_x, train_y)

In [12]:
def accuracy(y_true, y_pred):
  return (y_true == y_pred).mean()

train_acc = accuracy(train_y, qda.predict(train_x))
test_acc = accuracy(test_y, qda.predict(test_x))
print(f"Train (apparent) error is {1-train_acc:.4f} while test error is {1-test_acc:.4f}")

Train (apparent) error is 0.0111 while test error is 0.0500


# Consigna

### Implementación
1. Entrenar un modelo QDA utilizando ahora una *a priori* uniforme ¿Se observan diferencias?¿Por qué?
2. Implementar el modelo LDA, entrenarlo y testearlo contra los mismos sets que QDA ¿Se observan diferencias? ¿Podría decirse que alguno de los dos es notoriamente mejor que el otro?
3. Utilizar otros 2 (dos) valores de *random seed* para obtener distintos splits de train y test, y repetir la comparación del punto anterior ¿Qué se observa?
1. *(Opcional)* En `BaseBayesianClassifier._predict_one` se estima la posteriori de cada clase por separado, a pesar de que la cuenta es siempre la misma (cambiando valores de parámetros, pero no dimensiones). Aprovechando el *broadcasting* de NumPy, se puede reemplazar ese list comprehension por un cálculo *tensorizado* donde tanto medias como varianzas (o inversas) estén "stackeadas" sobre un tercer eje, permitiendo el cálculo en paralelo de todas las clases con un:
`log_posteriori = self.tensor_log_a_priori + self._predict_log_conditionals(x)`. Implementar dicha modificación y mostrar su uso. *Ayuda: los métodos `np.stack` y la documentación del operador [`@`](https://numpy.org/doc/stable/reference/generated/numpy.matmul.html) son de gran utilidad.*

### Preguntas técnicas

Responder las 7 preguntas que se encuentran distribuidas a lo largo del código.

### Preguntas teóricas

1. En LDA se menciona que la función a maximizar puede ser, mediante operaciones, convertida en:
$$
\log{f_j(x)} =  \mu_j^T \Sigma^{-1} (x- \frac{1}{2} \mu_j) + C'
$$
Mostrar los pasos por los cuales se llega a dicha expresión.

2. Explicar, utilizando las respectivas funciones a maximizar, por qué QDA y LDA son "quadratic" y "linear".

3. La implementación de QDA estima la probabilidad condicional utilizando 0.5*np.log(det(inv_cov)) -0.5 * unbiased_x.T @ inv_cov @ unbiased_x que no es exactamente lo descrito en el apartado teórico ¿Cuáles son las diferencias y por qué son expresiones equivalentes?


# Respuestas:
### <u> Preguntas teoricas </u>

#### 1- Derivación de LDA

Dado que para LDA $\Sigma_j == \Sigma $:
$$
\log{f_j(x)} =  -\frac{1}{2}\log |\Sigma| - \frac{1}{2} (x-\mu_j)^T \Sigma^{-1} (x- \mu_j) + C
$$

Ahora, como $-\frac{1}{2}\log |\Sigma|$ es común a todas las clases se puede incorporar a la constante aditiva. Luego, distribuyendo las matrices resultantes:
$$
\log{f_j(x)} =  -\frac{1}{2} x^T \Sigma^{-1} x + \frac{1}{2} x^T \Sigma^{-1} \mu_j + \frac{1}{2} \mu_j^T \Sigma^{-1} x - \frac{1}{2} \mu_j^T \Sigma^{-1} \mu_j + C 
$$

El primer término de la suma es independiente de la clase j, por lo que se puede incorporar a la constante aditiva.
Ahora, si tomamos el segundo término de la suma, usando la propiedad de la transpuesta de un producto de matrices $ (AB)^T = B^T A^T $ y que $ (A^T)^T = A $ , resulta que 
$$
x^T \Sigma^{-1} \mu_j = ( \mu_j^T (\Sigma^{-1})^T x )^T
$$
Dado que la matríz $\Sigma$ es cuadrada y simétrica, luego $\Sigma ^ -1$ es cuadrada y simétrica, entonces $ (\Sigma^{-1})^T = (\Sigma^{-1})$. Por lo tanto, la ecuación para el logaritmo de la probabilidad resulta:
$$
\log{f_j(x)} =  \frac{1}{2} [\mu_j^T \Sigma^{-1} x ]^T + \frac{1}{2} \mu_j^T \Sigma^{-1} x - \frac{1}{2} \mu_j^T \Sigma^{-1} \mu_j + C 
$$

Ahora, viendo las dimensiones de las matrices del primer término, tenemos que $\mu_j \epsilon ℝ^{(1xk)}$,  $\Sigma \epsilon ℝ^{(kxk)}$ y  $x \epsilon ℝ^{(kx1)} $, donde k es la cantidad de features del dataset.

Por lo tanto $ [\mu_j^T \Sigma^{-1} x ] $ es un escalar y la ecuación se puede reducir de la siguiente forma:
$$
\log{f_j(x)} =  \frac{1}{2} \mu_j^T \Sigma^{-1} x + \frac{1}{2} \mu_j^T \Sigma^{-1} x - \frac{1}{2} \mu_j^T \Sigma^{-1} \mu_j + C = \mu_j^T \Sigma^{-1} x - \frac{1}{2} \mu_j^T \Sigma^{-1} \mu_j + C = \mu_j^T \Sigma^{-1} (x - \frac{1}{2} \mu_j^T) + C
$$

Lo que resulta que, para LDA:
$$
\log{f_j(x)} = \mu_j^T \Sigma^{-1} (x - \frac{1}{2} \mu_j^T) + C
$$



#### 2- Quadratic vs Linear

Se sabe que la expresion a maximizar es

$$
\log{f_j(x)} = -\frac{1}{2}\log |\Sigma_j| - \frac{1}{2} (x-\mu_j)^T \Sigma_j^{-1} (x- \mu_j) + C
$$


Ahora, como $-\frac{1}{2}\log |\Sigma|$ es común a todas las clases se puede incorporar a la constante aditiva. Luego, distribuyendo las matrices resultantes:
$$
\log{f_j(x)} =  -\frac{1}{2} x^T \Sigma_j^{-1} x + \frac{1}{2} x^T \Sigma_j^{-1} \mu_j + \frac{1}{2} \mu_j^T \Sigma_j^{-1} x - \frac{1}{2} \mu_j^T \Sigma_j^{-1} \mu_j + C 
$$


En el caso de LDA El termino $-\frac{1}{2} x^T \Sigma^{-1} x $ es cuadratico en x, pero la matriz de covarianza NO depende de la clase (porque por hipotesis se supone que es la misma para todas las clases) por lo cual el termino es igual a todas las clases y puede considerarse como invariable, por lo que tambien puede incorporarse a la variable aditiva. Al incorporarse este termino a la constante aditiva, la ecuacion a maximizar solo tiene una componente lineal en x (como se ve en la expresion final de la pregunta teorica 1)

En el caso de QDA sin embargo el termino $-\frac{1}{2} x^T \Sigma_j^{-1} x $ es cuadratico en x, y ademas la matriz de covarianza depende de la clase por lo cual el termino entero varia por clase y no puede considerarse constante. Ese termino cuadratico en la ecuacion a maximizar es el que da a QDA su componente cuadratica.




#### 3-  Propiedades de los logaritmos

En la parte teórica, se presenta la siguiente fórmula para QDA:
$$
\log{f_j(x)} = -\frac{1}{2}\log |\Sigma_j| - \frac{1}{2} (x-\mu_j)^T \Sigma_j^{-1} (x- \mu_j) + C
$$
Sin embargo, el codigo provisto usa la inversa de matriz de covarianza: `0.5*np.log(det(inv_cov)) -0.5 * unbiased_x.T @ inv_cov @ unbiased_x`

Estas dos formas de calcular el resultado son equivalentes ya que, $ |\Sigma_j| = \frac{1}{|\Sigma_j^{-1}|} = (|\Sigma_j^{-1}|)^{-1} $ y por lo tanto el primer término de la fórmula teórica equivale a $  -\frac{1}{2}\log |\Sigma_j| = +\frac{1}{2}\log |\Sigma_j^{-1}| $

En la implementación práctica se prioriza utilizar la variable $ \Sigma_j^{-1} $ ya que es necesaria para calcular el segundo término y de esta forma se evita tener que guardar el valor tanto de $ \Sigma_j $ como de $ \Sigma_j^{-1} $.



### <u> Implementaciones </u>

#### 1) Entrenar un modelo QDA utilizando ahora una *a priori* uniforme ¿Se observan diferencias? ¿Por qué?

No se encontraron diferencias significativas al entrenar un modelo de QDA con una distribución uniforme en comparación con el modelo original provisto.

Esto se debe a que, en el caso de no proveer explicitamente una distribución a_priori cuando se llama al método QDA.fit(), la clase QDA computa automáticamente la distribución a_priori como la proporción de cada clase dentro del dataset. 
Para el caso del dataset "iris", el mismo se encuentra perfectamente balanceado, habiendo 50 muestras para cada clase. Por esta razón, el train_test_split() suele devolver dataset de train casi balanceados, por lo que el cálculo automático de QDA.fit() termina computando una distribución "casi" uniforme (ya que la proporción de cada clase dentro del dataset es la misma).

Aún así, si forzamos a que la distribución "a priori" sea uniforme, notamos que pueden existir diferencias respecto al código original aunque no son significativas.




In [13]:
qda = QDA()

a_priori_uniforme = np.array([1/3,1/3,1/3])

qda.fit(train_x, train_y,a_priori_uniforme)


def accuracy(y_true, y_pred):
  return (y_true == y_pred).mean()

train_acc = accuracy(train_y, qda.predict(train_x))
test_acc = accuracy(test_y, qda.predict(test_x))
print(f"Train (apparent) error is {1-train_acc:.4f} while test error is {1-test_acc:.4f}")

Train (apparent) error is 0.0111 while test error is 0.0500


#### 2) Implementar el modelo LDA, entrenarlo y testearlo contra los mismos sets que QDA ¿Se observan diferencias? ¿Podría decirse que alguno de los dos es notoriamente mejor que el otro?

Obsérvese que para ambos modelos, ajustarlos a los datos implica estimar los parámetros $(\mu_j, \Sigma_j) \; \forall j = 1, \dots, k$ en el caso de QDA, y $(\mu_j, \Sigma)$ para LDA.

Estos parámetros se estiman por máxima verosimilitud, de manera que los estimadores resultan:

* $\hat{\mu}_j = \bar{x}_j$ el promedio de los $x$ de la clase *j*
* $\hat{\Sigma}_j = s^2_j$ la matriz de covarianzas estimada para cada clase *j*
* $\hat{\pi}_j = f_{R_j} = \frac{n_j}{n}$ la frecuencia relativa de la clase *j* en la muestra
* $\hat{\Sigma} = \frac{1}{n} \sum_{j=1}^k n_j \cdot s^2_j$ el promedio ponderado (por frecs. relativas) de las matrices de covarianzas de todas las clases. *Observar que se utiliza el estimador de MV y no el insesgado*

Es importante notar que si bien todos los $\mu, \Sigma$ deben ser estimados, la distribución *a priori* puede no inferirse de los datos sino asumirse previamente, utilizándose como entrada del modelo.

In [14]:

class LDA(BaseBayesianClassifier):

  def _fit_params(self, X, y):
    # estimate COMMON  covariance matrix
    self.inv_covs = [inv(np.cov(X[:,y.flatten()==idx], bias=True)) 
                      for idx in range(len(self.log_a_priori))]
    

    total = len(y)  # ejemplo 90
    (unique, counts) = np.unique(y, return_counts=True)  # (0, cantidad de ceros) (1, cantidad de 1) (2, cantidad de 2)
    self.inv_cov_inbetween = np.zeros(self.inv_covs[0].shape)

    #asumo que las matrices de covariaza estan ordenadas por clase. entonces
    for index in range(len(counts)):
      self.inv_cov_inbetween = self.inv_cov_inbetween + np.multiply( self.inv_covs[index], (counts[index])/total )

    print(np.shape(self.inv_cov_inbetween))
    self.inv_covs = [self.inv_cov_inbetween for i in range(len(unique))]



    self.means = [X[:,y.flatten()==idx].mean(axis=1, keepdims=True) 
                  for idx in range(len(self.log_a_priori))]



  def _predict_log_conditional(self, x, class_idx):
    # predict the log(P(x|G=class_idx)), the log of the conditional probability of x given the class
    # this should depend on the model used
    inv_cov = self.inv_covs[class_idx]
    unbiased_x =  x - self.means[class_idx]
    return 0.5*np.log(det(np.linalg.inv( inv_cov))) -0.5 * unbiased_x.T @ inv_cov @ unbiased_x

In [15]:
un_lda = LDA()

un_lda.fit(train_x, train_y)

(4, 4)


#### 3) Utilizar otros 2 (dos) valores de *random seed* para obtener distintos splits de train y test, y repetir la comparación del punto anterior ¿Qué se observa?

In [16]:
def accuracy_qda(y_true, y_pred):
  return (y_true == y_pred).mean()

train_acc_q = accuracy_qda(train_y, qda.predict(train_x))
test_acc_q = accuracy_qda(test_y, qda.predict(test_x))
print(f"QDA Train (apparent) error is {1-train_acc_q:.8f} while test error is {1-test_acc_q:.8f}")


def accuracy_lda(y_true, y_pred):
  return (y_true == y_pred).mean()

train_acc_l = accuracy_lda(train_y, un_lda.predict(train_x))
test_acc_l = accuracy_lda(test_y, un_lda.predict(test_x))
print(f"LDA Train (apparent) error is {1-train_acc_l:.8f} while test error is {1-test_acc_l:.8f}")

QDA Train (apparent) error is 0.01111111 while test error is 0.05000000
LDA Train (apparent) error is 0.01111111 while test error is 0.03333333


## 3. Utilizar otros 2 (dos) valores de *random seed* para obtener distintos splits de train y test, y repetir la comparación del punto anterior ¿Qué se observa?

### Random seed: 6543

QDA Train (apparent) error is 0.01111111 while test error is 0.01666667

LDA Train (apparent) error is 0.04444444 while test error is 0.0166666

### Random seed: 0

QDA Train (apparent) error is 0.01111111 while test error is 0.03333333

LDA Train (apparent) error is 0.01111111 while test error is 0.08333333

### Random seed: 42

QDA Train (apparent) error is 0.01111111 while test error is 0.01666667

LDA Train (apparent) error is 0.03333333 while test error is 0.01666667

### Random seed: 65
QDA Train (apparent) error is 0.01111111 while test error is 0.05000000

LDA Train (apparent) error is 0.01111111 while test error is 0.03333333


Observaciones
- El error aparente de QDA en train es independiente del random seed. 
- El error aparente de LDA en train es dependiente del random seed. 
Esto puede deberse a que dado que dependiendo como se haga la seleccion de puntos de muestra durante el split, las matrices de covarianza resultantes para cada clase   pueden ser mas o menos distintas y estra mas o menos balanceadas, en cuyo caso la suposicion de una identica matriz de covarianza entre clases obtenidas a travez de un promedio ponderado resulta en que no todas las poblaciones en train pueden clasificarse correctamente mediante la misma. 

- En test, no hay un claro modelo preferencial, en ocaciones QDA devuelve una mejor prediccion de datos de test, en ocaciones LDA. 


